In [ ]:
from datasets import load_dataset
import json
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from json_repair import repair_json
import pandas as pd
from prutils.prevaluation import PrEvaluation
import time
dataset = load_dataset("jingjietan/essays-big5", cache_dir="../datasets")

name = "essays_gemini"

In [4]:
testset = dataset['test']
testset = pd.DataFrame(testset)


In [36]:
import google.generativeai as genai
from google.ai.generativelanguage_v1beta.types import content
genai.configure(api_key="___")

def inference(input):

  print()
  # Create the model
  generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 40,
    "max_output_tokens": 8192,
    "response_schema": content.Schema(
      type = content.Type.OBJECT,
      enum = [],
      required = ["dimensions"],
      properties = {
        "dimensions": content.Schema(
          type = content.Type.OBJECT,
          enum = [],
          required = ["Openness", "Conscientiousness", "Extraversion", "Agreeableness", "Neuroticism"],
          properties = {
            "Openness": content.Schema(
              type = content.Type.OBJECT,
              enum = [],
              required = ["evidence", "label"],
              properties = {
                "evidence": content.Schema(
                  type = content.Type.STRING,
                ),
                "label": content.Schema(
                  type = content.Type.INTEGER,
                ),
              },
            ),
            "Conscientiousness": content.Schema(
              type = content.Type.OBJECT,
              enum = [],
              required = ["evidence", "label"],
              properties = {
                "evidence": content.Schema(
                  type = content.Type.STRING,
                ),
                "label": content.Schema(
                  type = content.Type.INTEGER,
                ),
              },
            ),
            "Extraversion": content.Schema(
              type = content.Type.OBJECT,
              enum = [],
              required = ["evidence", "label"],
              properties = {
                "evidence": content.Schema(
                  type = content.Type.STRING,
                ),
                "label": content.Schema(
                  type = content.Type.INTEGER,
                ),
              },
            ),
            "Agreeableness": content.Schema(
              type = content.Type.OBJECT,
              enum = [],
              required = ["evidence", "label"],
              properties = {
                "evidence": content.Schema(
                  type = content.Type.STRING,
                ),
                "label": content.Schema(
                  type = content.Type.INTEGER,
                ),
              },
            ),
            "Neuroticism": content.Schema(
              type = content.Type.OBJECT,
              enum = [],
              required = ["evidence", "label"],
              properties = {
                "evidence": content.Schema(
                  type = content.Type.STRING,
                ),
                "label": content.Schema(
                  type = content.Type.INTEGER,
                ),
              },
            ),
          },
        ),
      },
    ),
    "response_mime_type": "application/json",
  }

  model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    generation_config=generation_config,
  )

  chat_session = model.start_chat(
    history= [
          {
              "role": "user",
              "parts": (
                  "Analyze the user's personality using the Big 5 framework based on the given text.\n"
                  "Output the result in this JSON format:\n"
                  "{\n"
                  "  'dimensions': {\n"
                  "    'Openness': {'evidence': '', 'label': '0/1'},\n"
                  "    'Conscientiousness': {'evidence': '', 'label': '0/1'},\n"
                  "    'Extraversion': {'evidence': '', 'label': '0/1'},\n"
                  "    'Agreeableness': {'evidence': '', 'label': '0/1'},\n"
                  "    'Neuroticism': {'evidence': '', 'label': '0/1'}\n"
                  "  }\n"
                  "}\n\n"
                  "For each dimension, provide:\n"
                  "- evidence: Analysis of the text reflecting the trait. Give examples (content and/or tone) from the user's message. Avoid using punctuation\n"
                  "- label: Provide only a value (0 for low; or 1 for high) to indicate the traits.\n\n"
              )
          }
      ]
  )

  response = chat_session.send_message(content=input)
  return response.text

In [ ]:
from tqdm import tqdm

df = pd.DataFrame(columns=["id","o_evi", "o_type", "c_evi", "c_type", "e_evi", "e_type", "a_evi", "a_type", "n_evi", "n_type", "O", "C", "E", "A", "N"])

for _, line in tqdm(testset.iterrows()):
    custom_id = line["__index_level_0__"]

    while True:
        try:
            return_content = inference(line['text'])
            break
        except:
            time.sleep(60)


    O = testset[testset["__index_level_0__"] == int(custom_id)]["O"].values[0]
    C = testset[testset["__index_level_0__"] == int(custom_id)]["C"].values[0]
    E = testset[testset["__index_level_0__"] == int(custom_id)]["E"].values[0]
    A = testset[testset["__index_level_0__"] == int(custom_id)]["A"].values[0]
    N = testset[testset["__index_level_0__"] == int(custom_id)]["N"].values[0]

    try:
        return_content = json.loads(return_content)
        return_content = return_content["dimensions"]

        o_evi = return_content["Openness"]["evidence"]
        o_type = return_content["Openness"]["label"]
        c_evi = return_content["Conscientiousness"]["evidence"]
        c_type = return_content["Conscientiousness"]["label"]
        e_evi = return_content["Extraversion"]["evidence"]
        e_type = return_content["Extraversion"]["label"]
        a_evi = return_content["Agreeableness"]["evidence"]
        a_type = return_content["Agreeableness"]["label"]
        n_evi = return_content["Neuroticism"]["evidence"]
        n_type = return_content["Neuroticism"]["label"]

        #make to pandas
        df = df.append({"id": custom_id, "o_evi": o_evi, "o_type": o_type, "c_evi": c_evi, "c_type": c_type, "e_evi": e_evi, "e_type": e_type, "a_evi": a_evi, "a_type": a_type, "n_evi": n_evi, "n_type": n_type, "O": O, "C": C, "E": E, "A": A, "N": N}, ignore_index=True)
    except:
        # append the return_content to a txt file
        with open(name + '_error.txt', 'a') as f:
            f.write(str(return_content))
            f.write('\n\n')



In [52]:
df["o_type"] = df["o_type"].apply(lambda x: 1 if str(x) == "1" else 0 if str(x).upper() == "0" else -1).astype(int)
df["c_type"] = df["c_type"].apply(lambda x: 1 if str(x) == "1" else 0 if str(x).upper() == "0" else -1).astype(int)
df["e_type"] = df["e_type"].apply(lambda x: 1 if str(x) == "1" else 0 if str(x).upper() == "0" else -1).astype(int) 
df["a_type"] = df["a_type"].apply(lambda x: 1 if str(x) == "1" else 0 if str(x).upper() == "0" else -1).astype(int)
df["n_type"] = df["n_type"].apply(lambda x: 1 if str(x) == "1" else 0 if str(x).upper() == "0" else -1).astype(int)

#make int
df["o_type"] = df["o_type"].astype(int)
df["c_type"] = df["c_type"].astype(int)
df["e_type"] = df["e_type"].astype(int)
df["a_type"] = df["a_type"].astype(int)
df["n_type"] = df["n_type"].astype(int)

df["O"] = df["O"].astype(int)
df["C"] = df["C"].astype(int)
df["E"] = df["E"].astype(int)
df["A"] = df["A"].astype(int)
df["N"] = df["N"].astype(int)



In [41]:
# save to csv
df.to_csv(name + '.csv', index=False)

In [42]:
# #read from csv
# import pandas as pd
# df = pd.read_csv(name + '.csv')


In [ ]:
O = PrEvaluation("O")
O.push([df["o_type"].to_list()],[df["O"].to_list()])
O.print_performance()

C = PrEvaluation("C")
C.push([df["c_type"].to_list()],[df["C"].to_list()])
C.print_performance()

E = PrEvaluation("E")
E.push([df["e_type"].to_list()],[df["E"].to_list()])
E.print_performance()

A = PrEvaluation("A")
A.push([df["a_type"].to_list()],[df["A"].to_list()])
A.print_performance()

N = PrEvaluation("N")
N.push([df["n_type"].to_list()],[df["N"].to_list()])
N.print_performance()